<a href="https://colab.research.google.com/github/andrewgcodes/FalconStreaming/blob/main/Falcon7B_Instruct_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Falcon 7B Instruct with Streaming

You will likely need Google Colab Premium, but the free plan might work. Enable the A100 GPU!


Streaming by default prints out all words onto one line. You may want to copy and paste the output into a doc to see it better.

There is Beam Search at the end of this notebook which is pretty fun to watch!

In [ ]:
!pip install git+https://www.github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/accelerate
!pip install bitsandbytes
!pip install einops

from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/huggingface/transformers to /tmp/pip-req-build-gv84w_hp
  Running command git clone --filter=blob:none --quiet https://www.github.com/huggingface/transformers /tmp/pip-req-build-gv84w_hp
  Resolved https://www.github.com/huggingface/transformers to commit 02d255db260253d137f6c627108f04dafeee4e81
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.4 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.30.0.dev0-py3-none-any.whl size=7165938 sha256=af42ce56e661f7f599d0f0ed67e1b25967f8840a173b08e39bc1b53dfa89c84c
  Stored in directory: /tm

## Model loading

In [ ]:
model_path="tiiuae/falcon-7b-instruct"
  
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, load_in_4bit=True, device_map="auto")


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")

# Set your prompt

In [ ]:
input_text = "Tell a story about a magical fish in the UAE:"

# Basic Streaming (Greedy)

In [ ]:
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

next_input = input_ids
max_length = 80  # Change this to your desired output length. Too long could cause an OOM Out of Memory error!
current_length = input_ids.shape[1]

while True:
    if current_length >= max_length:  # Check if we've reached the length limit
        break

    output = model(next_input)
    next_token_logits = output.logits[:, -1, :]
    next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
    print(tokenizer.decode(next_token_id[0].cpu().tolist(), skip_special_tokens=True), end='', flush=True)

    next_input = torch.cat([next_input, next_token_id.to("cuda")], dim=-1)

    current_length += 1

    if next_token_id[0].item() == tokenizer.eos_token_id:
        break



Once upon a time, in the depths of the Arabian Gulf, there lived a magical fish. This fish was unlike any other fish in the sea, as it had the ability to grant wishes to those who found it. One day, a young fisherman named Ali stumbled upon the magical fish while fishing in the Gulf. The fish, sensing a

# Top-k

In [ ]:
import torch
import torch.nn.functional as F

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

next_input = input_ids
max_length = 100  # Change this to your desired output length
current_length = input_ids.shape[1]
k = 10  # Number of tokens to sample from. Adjust as necessary. Greater k = greater variability.

while True:
    if current_length >= max_length:  # Check if we've reached the length limit
        break

    output = model(next_input)
    next_token_logits = output.logits[:, -1, :]
    
    # Apply top-k sampling
    indices_to_remove = next_token_logits < torch.topk(next_token_logits, k)[0][..., -1, None]
    next_token_logits[indices_to_remove] = float('-inf')
    next_token_id = torch.multinomial(F.softmax(next_token_logits, dim=-1), num_samples=1)

    print(tokenizer.decode(next_token_id[0].cpu().tolist(), skip_special_tokens=True), end='', flush=True)

    next_input = torch.cat([next_input, next_token_id.to("cuda")], dim=-1)

    current_length += 1

    if next_token_id[0].item() == tokenizer.eos_token_id:
        break



Once upon a time in the UAE, a magical fish lived in a deep blue pool in the middle of the desert. The fish was said to have special powers and was very close to the ruler of the Emirate. One day while the Emir was out for a swim, he was approached by the magical fish. The fish asked the Emir to grant him one wish. The Emir, taken aback by the fish's

# Beam Search (with Streaming)

In [ ]:
import torch

def beam_search(input_ids, model, tokenizer, beam_size, max_length):
    beam = [(input_ids, 0)]  # Each element in the beam is a tuple (input_ids, score)

    for step in range(max_length):
        new_beam = []
        print(f"\n=== Step {step + 1} ===")

        for beam_id, (prev_input_ids, prev_score) in enumerate(beam):
            output = model(prev_input_ids)
            next_token_logits = output.logits[:, -1, :]
            next_token_probs = torch.softmax(next_token_logits, dim=-1)

            top_probs, top_indices = torch.topk(next_token_probs, beam_size)
            
            for i in range(beam_size):
                next_input_ids = torch.cat([prev_input_ids, top_indices[0][i].unsqueeze(0).unsqueeze(0)], dim=-1)
                score = prev_score + torch.log(top_probs[0][i]).item()
                
                new_beam.append((next_input_ids, score))

        beam = sorted(new_beam, key=lambda x: x[1], reverse=True)[:beam_size]

        # Print out the current state of each beam
        for beam_id, b in enumerate(beam):
            print(f"\nBeam {beam_id + 1} Score: {b[1]}")
            print(tokenizer.decode(b[0][0].cpu().tolist(), skip_special_tokens=True))

    return beam

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

beam_size = 5  # Adjust as necessary
max_length = 100  # Adjust as necessary

beam = beam_search(input_ids, model, tokenizer, beam_size, max_length)

# Print out the highest-scoring output
print("\n=== Final Output ===")
print(f"\nBest Beam Score: {beam[0][1]}")
print(tokenizer.decode(beam[0][0][0].cpu().tolist(), skip_special_tokens=True))



=== Step 1 ===

Beam 1 Score: -0.0469970703125
Tell a story about a magical fish in the UAE:


Beam 2 Score: -4.98828125
Tell a story about a magical fish in the UAE: The

Beam 3 Score: -5.48046875
Tell a story about a magical fish in the UAE: A

Beam 4 Score: -5.6875
Tell a story about a magical fish in the UAE: Once

Beam 5 Score: -5.85546875
Tell a story about a magical fish in the UAE: 

=== Step 2 ===

Beam 1 Score: -0.4893798828125
Tell a story about a magical fish in the UAE:
Once

Beam 2 Score: -1.4151611328125
Tell a story about a magical fish in the UAE:
In

Beam 3 Score: -3.5938720703125
Tell a story about a magical fish in the UAE:
There

Beam 4 Score: -4.1173095703125
Tell a story about a magical fish in the UAE:
Deep

Beam 5 Score: -4.6407470703125
Tell a story about a magical fish in the UAE:
A

=== Step 3 ===

Beam 1 Score: -0.53790283203125
Tell a story about a magical fish in the UAE:
Once upon

Beam 2 Score: -1.7947998046875
Tell a story about a magical fish in the 